In [229]:
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2 as cv 
from os.path import join
import pydicom as dicom
from skimage import exposure
import progressbar 
%matplotlib qt

In [325]:
#path of the images
path = r'D:\Dalhousie\datasets'
#folder = r'05 2023.07.30 NoFiducials\cropped'
folder = r'06 2023.07.30 WithFiducials\cropped'

saving_folder = r'06 2023.07.30 WithFiducials\LogGabor'

in_path=join(path,folder)
entries = os.listdir(in_path)
image_path = join(in_path,entries[-2]) #Selected image
savedimage_path = join(path,saving_folder)

In [326]:
img=cv.imread(image_path,0)
cv.imshow('Imagen original',img)

cv.waitKey(0)
img.shape

(809, 745)

In [222]:
#Function to create a frequency grid to create the filter.
def Grid2D(imag_shape):
    ysize,xsize = imag_shape #The grid needs to be the same size as the image
    ymid = ysize // 2
    xmid = xsize // 2
    
    if ysize % 2 == 0: #Condition to led with images with odd or even size.
        ymax = ymid - 1
    else:
        ymax = ymid

    if xsize % 2 == 0:
        xmax = xmid - 1
    else:
        xmax = xmid
    
    X,Y= np.meshgrid(np.arange(-xmid, xmax + 1),np.arange(-ymid, ymax + 1)) #Spatial domain of the grid
    X = np.fft.ifftshift(X)/xsize #Frecuency domain of the grid (also normalized)
    Y = np.fft.ifftshift(Y)/ysize
    
    return X,Y

In [223]:
def RiezFilter(img_shape): 
    u,v = Grid2D(img_shape)
    w = np.sqrt(u**2+v**2)
    w[0,0]=0.0001
    
    H1 = (1j * u)/ w
    H2 = (1j * v)/ w
    
    return H1,H2

def LogGaborFilter(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    #for i in range(len(wavelenght)):
        
    lamb = wavelenght #[i]
    w_0 = 2/(lamb*3)

    GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2)))#+((phi-phi_0)**2/(2*sig_phi**2)) 
    GabFilter[0,0] = 0

    if (img_shape[0]%2 == 0):
        GabFilter[img_shape[0]//2,:]=0

    if (img_shape[1]%2 == 0):
        GabFilter[:,img_shape[1]//2]=0

    #FilterBank[i,:,:] = GabFilter
    return GabFilter

def MonogenicSignal(image,LogGaborFilt,H1,H2):
    
    F = np.fft.fft2(image)
    Ffilt = np.multiply(F,LogGaborFilt)
    
    F_g = np.fft.ifft2(Ffilt)
    
    F_gi = np.real(F_g)
    fm1 = np.real(F_g)
    
    F_modd_1 = np.multiply(Ffilt,H1) 
    F_modd_1 = np.fft.ifft2(F_modd_1)
    fm2 = np.real(F_modd_1)
    #print(np.min(np.imag(F_modd_1)))
    
    F_modd_2 = np.multiply(Ffilt,H2) 
    F_modd_2 = np.fft.ifft2(F_modd_2)
    fm3 = np.real(F_modd_2)
    
    return fm1,fm2,fm3


def FeatureSymetry(m1,m2,m3):
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    T = np.exp(np.log(np.mean(np.sqrt(odd**2+even**2))))
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    #FS = np.mean(FS, axis=0)
    #FA = np.mean(FA, axis=0)
    
    return FS,FA

H1,H2 = RiezFilter(img.shape)
wl = 36

lG = LogGaborFilter(img.shape,1.5,wl,0.55)

m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)

#LocalAmp = np.sqrt(m1**2+m2**2+m3**2)
#LocalPhase = np.arctan2((np.sqrt(m2**2+m3**2)),m1)

FS,FA = FeatureSymetry(m1,m2,m3)

FA = (FA-FA.min())*(255/(FA.max()-FA.min()))
FA = FA.astype(np.uint8)
FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
FS = FS.astype(np.uint8)


cv.imshow('',np.c_[img,FA,FS])
cv.waitKey(0)


-1

In [224]:
maximumvalue = len(entries)-1


bar = progressbar.ProgressBar(maxval=maximumvalue).start()

for ith in range (3,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])

    img = cv.imread(in_image_path,0)
    
    H1,H2 = RiezFilter(img.shape)
    wl = 60

    lG = LogGaborFilter(img.shape,1.5,wl,0.25)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS,FA = FeatureSymetry(m1,m2,m3)

    FA = (FA-FA.min())*(255/(FA.max()-FA.min()))
    FA = FA.astype(np.uint8)
    FS = (FS-FS.min())*(255/(FS.max()-FS.min()))
    FS = FS.astype(np.uint8)


    cv.imshow('',np.c_[img,FA,FS])
    cv.waitKey(1)
    #cv.imwrite(out_image_path,FS)
    bar.update(ith)

 99% (482 of 483) |##################### | Elapsed Time: 0:01:16 ETA:   0:00:00

# Multi-scale appoach

In [217]:
def LogGaborFilter_MS(img_shape,scaling_factor,wavelenght,k_w0):
    
    #Create a Frequency Grid
    X,Y = Grid2D(img_shape)
    
    FiltBank = np.zeros((len(wavelenght),img_shape[0],img_shape[1]))
    
    #Determine the spatial regions to use 
    w = np.sqrt(Y**2+X**2)
    w[0,0]=0.0001 #Avoids log(0)
    phi = np.arctan2(Y,X)
    
    #User defined parameter
    m = scaling_factor
    
    for i in range(len(wavelenght)):
        
        lamb = wavelenght[i]
        w_0 = 2/(lamb*3)

        GabFilter = np.exp( -((np.log(w/w_0))**2 / (2*(np.log(k_w0))**2)))#+((phi-phi_0)**2/(2*sig_phi**2)) 
        GabFilter[0,0] = 0

        if (img_shape[0]%2 == 0):
            GabFilter[img_shape[0]//2,:]=0

        if (img_shape[1]%2 == 0):
            GabFilter[:,img_shape[1]//2]=0

        FiltBank[i,:,:] = GabFilter
        
    return FiltBank

def FeatureSymetry_MS(m1,m2,m3):
    epsilon = 0.01
    
    odd = np.sqrt(m2**2 + m3**2)
    even = np.abs(m1)
    
    T = np.exp(np.log(np.mean(np.sqrt(odd**2+even**2))))
    
    denominator = np.sqrt(even**2+odd**2)+epsilon
    
    FS_num = np.clip((even - odd - T), 0,None)
    FA_num = np.clip((odd - even - T), 0,None)
    
    FS = FS_num/denominator
    FA = FA_num/denominator
    
    FS = np.sum(FS, axis=0)
    FA = np.sum(FA, axis=0)
    
    return FS,FA

H1,H2 = RiezFilter(img.shape)
wl = np.array([30,60,90,120])

lG = LogGaborFilter_MS(img.shape,1.5,wl,0.55)

m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
FA_m = FA_m.astype(np.uint8)
FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
FS_m = FS_m.astype(np.uint8)


cv.imshow('',np.c_[img,FA,FS,FA_m,FS_m])
cv.waitKey(0)

-1

In [327]:
maximumvalue = len(entries)-3


bar = progressbar.ProgressBar(maxval=maximumvalue).start()

for ith in range (3,maximumvalue):
    
    in_image_path = join(in_path,entries[ith])
    out_image_path = join(savedimage_path,entries[ith])
    
    img = cv.imread(in_image_path,0)
    
    H1,H2 = RiezFilter(img.shape)
    wl = np.array([70,80,90])

    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)


    cv.imshow('',np.c_[img,FS_m,FA_m])

    cv.imwrite(out_image_path,FS_m)
    cv.waitKey(1)
    bar.update(ith)

 99% (480 of 481) |##################### | Elapsed Time: 0:03:24 ETA:   0:00:00

# MRI images

In [323]:


maximumvalue = 160


bar = progressbar.ProgressBar(maxval=maximumvalue).start()

for ith in range (3,maximumvalue):
    I = dicom.dcmread('C:/Users/eddea/Desktop/Tesis Maestría/Codes/Original/'+str(ith)+'.dcm')
    I=I.pixel_array
    I=exposure.equalize_adapthist(I)
    img = I
    
    #img = cv.imread(in_image_path,0)
    
    H1,H2 = RiezFilter(img.shape)
    wl = np.array([5,10,15])

    lG = LogGaborFilter_MS(img.shape,1.5,wl,0.15)

    m1,m2,m3 = MonogenicSignal(img,lG,H1,H2)


    FS_m,FA_m = FeatureSymetry_MS(m1,m2,m3)

    FA_m = (FA_m-FA_m.min())*(255/(FA_m.max()-FA_m.min()))
    FA_m = FA_m.astype(np.uint8)
    FS_m = (FS_m-FS_m.min())*(255/(FS_m.max()-FS_m.min()))
    FS_m = FS_m.astype(np.uint8)


    cv.imshow('',np.c_[img,FS_m,FA_m,FS_m-FA_m,-FS_m+FA_m])

    #cv.imwrite(out_image_path,FS_m)
    cv.waitKey(1)
    bar.update(ith)

 98% (158 of 160) |##################### | Elapsed Time: 0:00:11 ETA:   0:00:00

In [ ]:
#Fussion 360